# Planning a surf trip using IOOS EDS Model Viewer

Created: 2022-11-25

Updated: 2022-11-25

Author: [Mathew Biddle](mailto:mathew.biddle@noaa.gov)

1. Look at significant wave height forecast for MD coast (38.2101 °N 75.0867 °W). Look for significant wave heights >1m or some significant increase to narrow down the investigation. This will get you to a 12-24 hour window typically.
   * Resources in EDS include:
     * Wave Watch III (WW3) WW3 Wave Hgt @ NOAA WW3 Wave Height
     * GFS-Wave WW3 Atlantic Height @ NOAA GFS-Wave Atlantic Wave Height https://eds.ioos.us/thredds/catalog/ioos/gfswave/global/catalog.html
     * The Nearshore Wave Prediction System (NWPS) Chesapeake NWPS AKQ Wave Hgt @ NOAA NWPS Chesapeake Wave Height - https://eds.ioos.us/thredds/catalog/ioos/nwps/akq/catalog.html
1. Once you find a blip in wave heights, take a look at wave periods in that time period to see if it's just a storm or a decent offshore swell. Typically periods >5 seconds is a good start. That should get you down to a 12 hour window where it might be decent to surf.
1. Next look at the forecasted wave direction and wind direction/speed. Lots of nuances in these two pieces as wind direction could be bad (onshore) but if speeds are low it could be alright. If wind direction good (offshore) but winds are strong, could be tricky. wave direction informs which spots would catch the swell. Not many features on MD coast (barrier island), so direction not as important. This should get you to a 6 hour'ish window.
1. Finally, look at tides and water temp (wetsuit or trunks?), sunrise, sunset times.
1. Run this process every hour leading up to the event - forecast will get more accurate the closer to the event. I will start looking at real-time buoy obs ~12 hours before the event.

In [2]:
from siphon.catalog import TDSCatalog
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

# GFS WW3

In [6]:
catUrl = "https://eds.ioos.us/thredds/catalog/ioos/gfswave/global/latest.xml"
datasetName = "GFS Wave Global 25km-2022-11-25"

### Access a dataset
With the TDS catalog url, we can use Siphon to get the dataset named `datasetName`.

In [7]:
catalog = TDSCatalog(catUrl)

In [8]:
catalog.datasets

['GFS Wave Global 25km-2022-11-25', 'gfswave.global.0p25.2022112506.f384.grib2', 'gfswave.global.0p25.2022112506.f381.grib2', 'gfswave.global.0p25.2022112506.f378.grib2', 'gfswave.global.0p25.2022112506.f375.grib2', 'gfswave.global.0p25.2022112506.f372.grib2', 'gfswave.global.0p25.2022112506.f369.grib2', 'gfswave.global.0p25.2022112506.f366.grib2', 'gfswave.global.0p25.2022112506.f363.grib2', 'gfswave.global.0p25.2022112506.f360.grib2', 'gfswave.global.0p25.2022112506.f357.grib2', 'gfswave.global.0p25.2022112506.f354.grib2', 'gfswave.global.0p25.2022112506.f351.grib2', 'gfswave.global.0p25.2022112506.f348.grib2', 'gfswave.global.0p25.2022112506.f345.grib2', 'gfswave.global.0p25.2022112506.f342.grib2', 'gfswave.global.0p25.2022112506.f339.grib2', 'gfswave.global.0p25.2022112506.f336.grib2', 'gfswave.global.0p25.2022112506.f333.grib2', 'gfswave.global.0p25.2022112506.f330.grib2', 'gfswave.global.0p25.2022112506.f327.grib2', 'gfswave.global.0p25.2022112506.f324.grib2', 'gfswave.global.0p2

In [9]:
ds = catalog.datasets[datasetName]
ds.name

'GFS Wave Global 25km-2022-11-25'

Datasets each have a set of access protocols:

In [19]:
ds.access_urls

{'HTTPServer': 'https://eds.ioos.us/thredds/fileServer/ioos/gfswave/global/GFS_Wave_Global_25km-2022-11-25'}

In [10]:
list(ds.access_urls)

['HTTPServer']

Siphon's `remote-access` returns a `Dataset` object, which opens the remote dataset and provides access to its metadata:

In [11]:
dataset = ds.remote_access()

ValueError: OPENDAP is not available for this dataset

In [ ]:
list(dataset.ncattrs())

### Display a variable:
1) Create a widget to select from a list of all variables in this dataset.


In [ ]:
var_name = widgets.RadioButtons(
    options=list(dataset.variables),
    description='Variable:')


2) Run the cell below to display the widget.  
3) Select the variable you wish to view.

In [ ]:
display(var_name)

4) Display information about the chosen variable

In [ ]:
var = dataset.variables[var_name.value];
print("Name: " + var.name);
print("Dimensions: " + str(list(var.dimensions)));
print("Shape: " + str(var.shape));
import operator
from functools import reduce
nelems = reduce(operator.mul, var.shape, 1);
print("# elements: " + str(nelems));
print("Datatype: " + str(var.dtype));

5) Attempt to plot the variable.

In [ ]:
# EDIT these values to print or plot fewer/more elements
max_print_elems = 1000; # don't print more than this number of elements
max_elems = 10000; # don't request more than this number of elements (<100M to avoid HTTP errors)

# only atttempt to plot numeric types
if (not(var.dtype == np.uint8 or np.can_cast(var.dtype, float, "same_kind"))):
    print("Not a numeric type - cannot plot variable: ", var.name);
    if (nelems > max_print_elems):
        print("Too many elements - printing first " + str(max_print_elems) + " elements");
        var = var.flatten()[0:max_print_elems];
        print(var);
        
else:
    # assure plotable number of dimensions
    ndims = len([s for s in var.shape if s > 1]);
    max_dims = 2;
    shape = np.array(var.shape);
    if (ndims > max_dims):
        print("Too many dimensions - reducing last " + str(ndims-max_dims) + " dimensions.")
        shape[np.argwhere(shape>1).flatten().tolist()[max_dims:]] = 1;
        print("New shape: " + str(shape))
        ndims = max_dims;

    # assure plotable number of elements   
    nelems = reduce(operator.mul, shape, 1);
    scale = (nelems/max_elems)**(1/ndims) if ndims else 0;
    if (scale > 1):
        print("Too many elements - subsetting variable")
        shape[np.argwhere(shape>1).flatten().tolist()] = shape[np.argwhere(shape>1).flatten().tolist()]//scale;
        print("New shape: " + str(shape));
              
    # EDIT indices variable to change the number/indices of plotted elements
    to_slice = lambda x : slice(None, x, None)
    indices = [to_slice(s) for s in shape]
    
    # make copy of (possible) subset of variable
    disp_var = var[indices];

    # plot the variable
    %matplotlib inline
    # for one-dimensional data, print value
    if (ndims == 0):
        print(var.name, ": ", disp_var)
    # for two-dimensional data, make a line plot
    elif (ndims == 1):
        plt.plot(np.squeeze(np.array([range(len(np.squeeze(disp_var[:])))])), np.squeeze(disp_var[:]), 'bo', markersize=5)
        plt.title(var.name)
        plt.show()
    # for three-dimensional data, make an image
    elif (ndims == 2):
        plt.imshow(np.squeeze(disp_var[:]))
        plt.title(var.name)
        plt.show()

**Note** that data are only transferred over the network when the variable is sliced, and only data corresponding to the slice are downloaded. In this case, we are ask for a subset of the data with `disp_var = var[indices]`. You may change the values to `indices` to request a different subset of data.

---

6) To plot a different variable, select it in the widget and rerun the subsequent cells.

### More with Siphon
To see what else you can do, view the [Siphon API](https://unidata.github.io/siphon/latest/api/index.html).

In [ ]:
### Your code here ###